In [1]:
# -*- coding: utf-8 -*-
"""
Created on Sun Apr  05 20:20:02 2020
爬虫脚本：佛山链家房源信息
@author: yihao.he
"""
from bs4 import BeautifulSoup  
import pandas as pd
from tqdm import tqdm
import math
import requests  
import lxml
import re
import time




area_dic = {'禅城区':'chancheng',
            '南海区':'nanhai',
            '顺德区':'shunde',
            '三水区':'sanshui1',
            '高明区':'gaoming1'}


# 加个header以示尊敬
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/65.0.3325.146 Safari/537.36',
           'Referer': 'https://fs.lianjia.com/ershoufang/'}

# 新建一个会话
sess = requests.session()
sess.get('https://fs.lianjia.com/ershoufang/', headers=headers)

# url示例：https://fs.lianjia.com/ershoufang/luohuqu/pg2/
url = 'https://fs.lianjia.com/ershoufang/{}/pg{}/'




# 当正则表达式匹配失败时，返回默认值（errif）
def re_match(re_pattern, string, errif=None):
    try:
        return re.findall(re_pattern, string)[0].strip()
    except IndexError:
        return errif



# 新建一个DataFrame存储信息
data = pd.DataFrame()

for key_, value_ in area_dic.items():
    # 获取该行政区下房源记录数
    start_url = 'https://fs.lianjia.com/ershoufang/{}/'.format(value_)
    html = sess.get(start_url).text
    house_num = re.findall('共找到<span> (.*?) </span>套.*二手房', html)[0].strip()
    print('💚{}: 二手房源共计「{}」套'.format(key_, house_num))
    time.sleep(1)
    # 页面限制🚫 每个行政区只能获取最多100页共计3000条房源信息
    total_page = int(math.ceil(min(3000, int(house_num)) / 30.0))
    for i in tqdm(range(total_page), desc=key_):
        html = sess.get(url.format(value_, i+1)).text
        soup = BeautifulSoup(html, 'lxml')
        info_collect = soup.find_all(class_="info clear")
        
        for info in info_collect:
            info_dic = {}
            # 行政区
            info_dic['area'] = key_
            # 房源的标题
            info_dic['title'] = re_match('target="_blank">(.*?)</a><!--', str(info))
            # 小区名
            info_dic['community'] = re_match('xiaoqu.*?target="_blank">(.*?)</a>', str(info))
            # 位置
            info_dic['position'] = re_match('<a href.*?target="_blank">(.*?)</a>.*?class="address">', str(info))
            # 税相关，如房本满5年
            info_dic['tax'] = re_match('class="taxfree">(.*?)</span>', str(info))
            # 总价
            info_dic['total_price'] = float(re_match('class="totalPrice"><span>(.*?)</span>万', str(info)))
            # 单价
            info_dic['unit_price'] = float(re_match('data-price="(.*?)"', str(info)))
            
            # 匹配房源标签信息，通过|切割
            # 包括面积，朝向，装修等信息
            icons = re.findall('class="houseIcon"></span>(.*?)</div>', str(info))[0].strip().split('|')
            info_dic['hourseType'] = icons[0].strip()
            info_dic['hourseSize'] = float(icons[1].replace('平米', ''))
            info_dic['direction'] = icons[2].strip()
            info_dic['fitment'] = icons[3].strip()
            
            # 存入DataFrame
            if data.empty:
                data = pd.DataFrame(info_dic,index=[0])
            else:
                data = data.append(info_dic,ignore_index=True)




# 去掉一条面积10000+平米的房源记录
data = data[data['hourseSize'] < 10000]
print(data.head())



💚禅城区: 二手房源共计「6827」套


禅城区: 100%|███████████████████████████████████████████████████████████████████████████| 100/100 [01:44<00:00,  1.12s/it]


💚南海区: 二手房源共计「13692」套


南海区: 100%|███████████████████████████████████████████████████████████████████████████| 100/100 [01:59<00:00,  1.30s/it]


💚顺德区: 二手房源共计「9127」套


...

💚三水区: 二手房源共计「2316」套


三水区: 100%|█████████████████████████████████████████████████████████████████████████████| 78/78 [01:42<00:00,  1.17s/it]


💚高明区: 二手房源共计「557」套


高明区: 100%|█████████████████████████████████████████████████████████████████████████████| 19/19 [00:26<00:00,  1.33s/it]


  area                      title  community position    tax  total_price  \
0  禅城区  急售！保利香槟花园 满五唯一 精装修刚需轻奢大三房     保利香槟花园       张槎   None        148.0   
1  禅城区            满5唯一税费少 小区环境好又靓       翠影华庭       张槎  房本满五年         70.0   
2  禅城区     东海银湾精装3房 满五年税少 总价低 快上车     长信东海银湾       朝安   None        170.0   
3  禅城区        智博丽海花园 高楼层东南向 带装修出售  联诚·智博丽海花园       石湾   None        152.0   
4  禅城区       东南向精致3房， 安静望花园  随时看房       时代年华       澜石   None        160.0   

   unit_price hourseType  hourseSize direction fitment  
0     16323.0       3室2厅       90.67         北      精装  
1     10494.0       2室1厅       66.71         南      简装  
2     20988.0       3室1厅       81.00         北      精装  
3     16766.0       3室2厅       90.66        东南      精装  
4     19278.0       3室1厅       83.00         南      精装  


In [2]:
#保存到 CSV
data.to_csv('.\\JN_DataWarehouse\\foshan_lianjiafangyuan\\lianjia-foshanfangyuan.csv')